# Лабораторная работа №1

## Задание 1

Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).

In [2]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [3]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

In [6]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

In [8]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})
X

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
145058,0,14.6,23.9,0.0,0,24.0,0,0,6.0,9.0,52.0,30.0,1019.6,1016.8,16.4,23.2,False,False
86228,1,21.2,34.2,2.6,1,37.0,1,0,7.0,11.0,29.0,58.0,1010.3,1009.2,33.9,28.1,True,False
120792,2,12.2,32.7,0.0,2,28.0,0,1,9.0,7.0,55.0,23.0,1009.3,1004.9,20.7,31.9,False,False
40076,3,10.9,19.7,6.6,3,33.0,2,2,9.0,9.0,98.0,50.0,1035.9,1033.2,13.3,18.7,True,False
47664,4,3.5,26.1,0.0,4,81.0,3,3,2.0,41.0,71.0,26.0,1011.7,1006.4,12.6,25.8,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61738,30,4.6,17.8,0.0,6,46.0,15,10,9.0,28.0,82.0,44.0,1016.5,1015.7,11.1,16.8,False,False
3794,6,20.5,36.1,0.0,10,33.0,11,15,9.0,9.0,70.0,45.0,1008.8,1006.5,23.2,33.1,False,False
88580,28,20.0,31.1,0.0,0,33.0,0,0,13.0,17.0,58.0,63.0,1014.6,1011.7,28.2,29.2,False,False
70898,35,4.1,20.1,0.0,7,20.0,14,1,6.0,9.0,70.0,34.0,1023.6,1020.7,9.4,19.5,False,False


In [10]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

In [12]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 2

Провести классификацию найденного датасета, методом к-ближайших соседей. 
В формате Markdown писать пояснения. 
Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [33]:
from sklearn.neighbors import KNeighborsClassifier

In [35]:
kn_classifier = KNeighborsClassifier()
kn_params ={
    'n_neighbors': range(1,30),
    'metric':['minkowski','euclidean','manhattan']  
}
kn_grid = GridSearchCV(kn_classifier, kn_params, cv=5, n_jobs=-1)
kn_grid.fit(X_train, y_train)
kn_best_params = kn_grid.best_params_

In [36]:
kn_best_model = KNeighborsClassifier(**kn_best_params)
kn_best_model.fit(X_train, y_train)
kn_predicted = kn_best_model.predict(X_test)
print('Used params:',kn_best_params)
print('Evaluation:\n', metrics.classification_report(y_test, kn_predicted,digits=5))

Used params: {'metric': 'manhattan', 'n_neighbors': 21}
Evaluation:
               precision    recall  f1-score   support

       False    0.84895   0.96245   0.90214      9028
        True    0.75940   0.40902   0.53168      2616

    accuracy                        0.83811     11644
   macro avg    0.80418   0.68574   0.71691     11644
weighted avg    0.82883   0.83811   0.81891     11644

